In [1]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
print(n)
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()

"""Creating tuples"""
from itertools import product

import timeit

928875724


In [ ]:
"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(728,729):
    if len(gap.IdsOfAllSmallGroups(i+1)) < 1500:
        small += gap.IdsOfAllSmallGroups(i+1)
    else:
        print(i+1)
# small += gap.IdsOfAllSmallGroups(96)

In [2]:
"""For original problem with only trivial character."""
class induce():
    """Function to return list of class sizes in default order."""
    def __init__(self):
        pass

    """Returns class sizes up to conjugacy"""
    def class_sizes(self, G):
        group_order = G.order()
        reps = G.conjugacy_classes_representatives()
        class_sizes = []
        for g in reps:
            class_sizes.append(group_order / G.centralizer(g).order())
#         print(class_sizes)
        return class_sizes

    """Function to find all subgroups. Returns list of subgroups."""
    def subgroups(self, G):
        subgroups = G.conjugacy_classes_subgroups()
#         print(subgroups)
        return subgroups

    """Function to find induced trivial character of each subgroup. Returns a list of lists."""
    def get_induced(self, subgroups, G):
        ind = []
        for subgroup in subgroups:
            ind.append(subgroup.trivial_character().induct(G))
#         print(ind)
        return ind

    """Forms the matrices that we use to solve the simultaneous equation later. 
    matrix[1:-1]: Excludes the first and last subgroup (trivial subgroup and the group itself), and the first column.
    checkmat[1:-1] Used for checking value at identity."""
    def mat(self, induced):
        matrix = []
        for char in induced:
            matrix.append(list(char.values()))
        neg = []
        for i in range(len(matrix[0])):
            neg.append(-1)
        matrix[-1] = neg
#         for line in matrix:
#             print(line)
        return matrix[1:]

    """Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise."""
    def test(self, G):
        a = self.class_sizes(G)
        b = self.subgroups(G)
        c = self.get_induced(b, G)
        d = self.mat(c)
        e = Matrix(d).T
        temp = []
        temp.append(G.order())
        for i in range(len(a)-1):
            temp.append(0)
        Y = vector(temp)
        p = MixedIntegerLinearProgram(solver='GLPK')
        """Set integer boolean for integer or rational solutions."""
        w = p.new_variable(integer = True, nonnegative = True)
        p.add_constraint(e*w == Y)
        try:
            p.solve()
            """Prints group and SmallGroup Id."""
#             print(G)
            print(str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]))
            
            """Tape is used for finding representation decomposition"""
#             tape = [0]
#             for i, v in (p.get_values(w).items()):
#                 if round(v,6) != 0:
#                     tape.append(int(round(v)))
#                 else:
#                     tape.append(0)
#             return tape
                    
            """Prints the structure of the decomposition."""
            out = []    
            for i, v in (p.get_values(w).items()):
                out.append('w_%s = %s' % (i, v))
                """Prints subgroup and subgroup order."""
                if round(v,6) != 0:
                    print("Order: " + str(b[i+1].order()) + ", Coeff: " + str(round(v)))
                    print(d[i])
            """Prints the solution"""
            print(out)
        except:
            pass
#             print('fail')

In [3]:
def repDecom(G,inp,inducer):
#     """Define the group here"""
#     G = gap.SmallGroup([36,11])

    """Create the Group Ring and Embedding. All future references of elements of group algebra must call emb"""
    R = gap.Rationals
    GA = gap.GroupRing(R,G)
    emb = gap.Embedding(G,GA)
    elements = gap.Elements(G)
    
    """Subgroups and the generators"""
    cc = gap.ConjugacyClassesSubgroups(G)
    # for cl in cc:
    #     print(gap.Order(cl[1]))
    
    """Creating tuples for iteration. ONLY works for HpGroups with Integer Decomp"""
    nums = 0
    count = 0
    for ada in range(len(inp)-1):
        if inp[ada] != 0:
            nums = max(nums,len(gap.Elements(cc[ada+1])))
            if len(gap.Elements(cc[ada+1])) != 1:
                count += 1
    tuples = product(range(1,nums+1),repeat=count)
    
    for tup in tuples:
        i = 0
        subgroups = []
        for num in range(len(inp)-1):
            for index in range(inp[num]):
                if len(gap.Elements(cc[num+1])) == 1:
                    subgroups.append(gap.Elements(cc[num+1])[index+1])
                else:
                    subgroups.append(gap.Elements(cc[num+1])[tup[i]])  
                    i += 1
        gens = gap([])

        for subgroup in subgroups:
            sub_elems = gap.Elements(subgroup)
            temp = sub_elems[1]^emb
            for i in range(len(sub_elems)-1):
                temp += sub_elems[i+2]^emb
            gap.Add(gens, temp)

        ideal = gap.LeftIdeal(GA,gens)
        if gap.Dimension(ideal) == gap.Dimension(GA):
            print("Is Rep Decomposable")
        print(gap.Dimension(ideal))
    
    """Main method to run for all groups."""
#     subgroups = []
#     for num in range(len(inp)-1):
#         for index in range(inp[num]):
#             subgroups.append(gap.Elements(cc[num+1])[index+1])  

#     subgroups = [gap.Elements(cc[3])[1], 
#                  gap.Elements(cc[4])[3],
#                  gap.Elements(cc[5])[3],
#                     gap.Elements(cc[8])[1]]
#     gens = gap([])

#     for subgroup in subgroups:
#         sub_elems = gap.Elements(subgroup)
#         temp = sub_elems[1]^emb
#         for i in range(len(sub_elems)-1):
#             temp += sub_elems[i+2]^emb
#         gap.Add(gens, temp)
    
#     """Creating ideal and checking dimension"""
#     ideal = gap.LeftIdeal(GA,gens)
#     if gap.Dimension(ideal) == gap.Dimension(GA):
#         print("Is Rep Decomposable")
#     print(gap.Dimension(ideal))

In [5]:
"""Main function
If group is in id form.
https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
This works on CoCalc/non-windows system with gap_packages library installed."""
if __name__ == "__main__":
    inducer = induce()
    groups = [[243,26]]
    for groupid in groups:
        G = gap.SmallGroup(groupid)
#         tape = inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
#         print(tape)
#         repDecom(G.AsPermGroup(),tape,inducer)
        try:
            inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
        except:
            pass
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
#     groups = [PSL(2,32)]
        
#     for group in groups:
#         try:
#             inducer.test(group)
#         except:
#             pass

243,26
Order: 3, Coeff: 1.0
[81, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Order: 3, Coeff: 1.0
[81, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
Order: 3, Coeff: 1.0
[81, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Order: 81, Coeff: 1.0
[3, 0, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3]
Order: 243, Coeff: 3.0
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
['w_0 = 0.0', 'w_1 = 0.0', 'w_2 = 1.0', 'w_3 = 1.0', 'w_4 = 1.0', 'w_5 = 0.0', 'w_6 = 0.0', 'w_7 = 0.0', 'w_8 = 0.0', 'w_9 = 0.0', 'w_10 = 0.0', 'w_11 = 0.0', 'w_12 = 0.0', 'w_13 = 0.0', 'w_14 = 0.0', 'w_15 = 0.0', 'w_16 = 0.0', 'w_17 = 0.0', 'w_18 = 0.0', 'w_19 = 1.0', 'w_20 = 0.0', 'w_21 = 0.0', 'w_22 = 3.0']
